In [2]:
import requests
from bs4 import BeautifulSoup
import chardet
import pickle
from tqdm import tqdm
import numpy as np
import time

[국내 개발자 커뮤니티 리스트](https://zetawiki.com/wiki/%EA%B5%AD%EB%82%B4_%EA%B0%9C%EB%B0%9C%EC%9E%90_%EC%BB%A4%EB%AE%A4%EB%8B%88%ED%8B%B0)

우선 대표적인 자바 개발 커뮤니티 [OKKY](https://okky.kr/) 에 대한 크롤링을 진행했다


In [89]:
seed="https://okky.kr"
params = {
    "max":"100",
    "sort":"id",
    "order":"desc"
}
headers = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "Accept-Encoding":None #인코딩 문제 해결 위해
}

resp = requests.get(seed+"/articles/tech-qna",params=params, headers=headers)

In [90]:
chardet.detect(resp.content)

{'confidence': 0.99, 'encoding': 'utf-8', 'language': ''}

In [91]:
soup = BeautifulSoup(resp.content,'lxml')

In [92]:
h5 = soup.select('h5 > a')
h5[:5]

[<a href="/article/423650">
                 
                 OKKY Q&amp;A 운영 안내 - 질문 전 꼭 읽어보세요!
             </a>, <a href="/article/508699">
                 
                 리스너 호출 오류
             </a>, <a href="/article/508698">
                 
                 씨언어 잘 아시는분들.. 시간 나시면 코드짜기좀 봐주실수 있으신가요?
             </a>, <a href="/article/508693">
                 
                 jsonObject-&gt; Ajax 호출 질문입니다. 중복검사
             </a>, <a href="/article/508691">
                 
                 Angular와 React 궁금한것이 있습니다.
             </a>]

In [108]:
links = [el['href'] for el in h5] # resp에서 각 게시글로의 링크를 따온다
links[1:10]

['/article/508699',
 '/article/508698',
 '/article/508693',
 '/article/508691',
 '/article/508690',
 '/article/508688',
 '/article/508680',
 '/article/508650',
 '/article/508643']

함수로 만들어보자

In [113]:
def get_links(resp)->list: # 링크들을 가져와 리스트로 반환하는 함수
    try:
        soup = BeautifulSoup(resp.content,'lxml')
        h5 = soup.select('h5 > a')
        links = [el['href'] for el in h5[1:]]
        
    except Exception as e:
        print("Exception occurs in get_links",resp.url,e)
    return links

In [114]:
def get_QA(links:list): # 링크의 리스트를 받아 dict 인 ques와 ans에 데이터를 저장하는 함수
    def get_qid(soup,que):
        try:
            que.append(soup.find("span",{"class":"list-group-item-text article-id"}).text)
        except Exception as e:
            que.append(np.nan)
    def get_uid(soup,que):
        try:
            tmp = soup.find("span",{"class":"nickname"})
            if tmp==None:
                tmp = soup.find("a",{"class":"nickname"})
            que.append(tmp.text)
        except Exception as e:
            que.append(np.nan)        
    def get_tstamp(soup,que):
        try:
            que.append(soup.find("span",{"class":"timeago"}).text)
        except Exception as e:
            que.append(np.nan)  
    def get_tags(soup,que):
        try:
            que.append(tuple([tag.text for tag in soup.find_all("a",{"class":"list-group-item-text item-tag label label-gray"})]))
        except Exception as e:
            que.append(tuple())
    def get_view(soup,que):
        try:
            que.append(soup.find("div",{"class":"content-identity-count"}).text)
        except Exception as e:
            que.append(np.nan)
    def get_vote(soup,que):
        try:
            que.append(soup.find("div",{"class":"content-eval-count"}).text)
        except Exception as e:
            que.append(np.nan)
    def get_title(soup,que):
        try:
            que.append(soup.find("h2",{"class":"panel-title"}).text.strip())
        except Exception as e:
            que.append(np.nan)
    def get_content(soup,que):
        try:
            content = tuple([p.text.strip() for p in soup.select("#content-body > article > p")]) # p로 구분된 본문
            if content == tuple():
                content = soup.find("article",{"class":"content-text"}).text.strip()
            que.append(content)      
        except Exception as e:
            que.append(tuple())
    def get_code(soup,que):
        try:
            que.append(tuple([pre.text.strip() for pre in soup.select("#content-body > article > pre")]))
        except Exception as e:
            que.append(tuple())
    def get_uid_a(li):
        try:
            tmp = li.find("a",{"class":"nickname"})
            if tmp ==None:
                tmp = li.find("span",{"class":"nickname"})
            an.append(tmp.text)
        except Exception as e:
            an.append(np.nan)
    def get_tstamp_a(li):
        try:
            an.append(li.find("span",{"class":"timeago"}).text)
        except Exception as e:
            an.append(np.nan) 
    def get_vote_a(li):
        try:
            an.append(li.find("div",{"class":"content-eval-count"}).text)
        except Exception as e:
            an.append(np.nan)
    def get_comment(li):
        try:
            an.append(li.find("article",{"class":"list-group-item-text note-text"}).text.strip())
        except Exception as e:
            an.append(np.nan)
    
    
    for link in links:
        try:
            url = seed + link
            resp = requests.get(url, headers=headers,timeout=3)
            soup = BeautifulSoup(resp.content,'lxml')
            
            que = []
            
            get_qid(soup,que)
            get_uid(soup,que)
            get_tstamp(soup,que)
            get_tags(soup,que)
            get_view(soup,que)
            get_vote(soup,que)
            get_title(soup,que)   
            get_content(soup,que)    
            get_code(soup,que)   
            
            ques[que[0]] = tuple(que[1:])
            ans[que[0]] = {}
            
            lst_group = soup.find("ul",{"class":"list-group"})
            if lst_group==None:
                continue
            lis = lst_group.find_all("li",{"class","list-group-item note-item clearfix"})
            if lis == []:
                continue
            for i,li in enumerate(lis):
                an = []

                get_uid_a(li)
                get_tstamp_a(li)
                get_vote_a(li)
                get_comment(li) 
                
                # 게시글의 답글을 순서대로 id 부여한다. str(que[0])+'_'+str(i)로 지정하고 이를 키로 설정
                ans[que[0]][str(que[0])+'_'+str(i)] = tuple(an) 
   
        except Exception as e:
            print("Exception occurs in get_QA",link,e)
            continue

    

In [4]:
# 약 127500 건의 게시글을 가져온다
if __name__=="__main__":
    seed="https://okky.kr"
    params = {
        "max":"100", # 100개 게시글 한번에 조회
        "sort":"id", 
        "order":"asc",
        "offset":"0"
    }
    headers = {
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    }
    
    # 
    ques = {} # 데이터 저장 딕셔너리 ques = {que_id:(user_id, time_stamp, (tags), views, vote_count, title, content, code)}
    ans = {} # 데이터 저장 딕셔너리 ans = {que_id: {an_id:(user_id, time_stamp, vote_count, content)} }
    
    for offset in tqdm(range(0,127500,100)):
        try:
            params['offset'] = str(offset) 
            resp = requests.get(seed+"/articles/tech-qna",params=params, headers=headers)
            
            links = get_links(resp)
            
            get_QA(links)
            
            
        except Exception as e:
            print("Exception occurs in main",e)
            continue
        
        finally:
            with open('questions.dic','wb') as fp: # 질문 데이터를 dict 형태로 저장
                pickle.dump(ques,fp)
            with open('answers.dic','wb') as fp: # 답글 데이터를 dict 형태로 저장
                pickle.dump(ans,fp)

In [5]:
with open('questions.dic','rb') as fp:
    questions = pickle.load(fp)

In [6]:
questions['#508581'] # #508581 key로 value 값인 튜플 조회

('dokdo_---',
 '2018-10-07 00:16:03',
 (),
 ' 3',
 '0',
 '프로그래머는 자격증이 필수인가요??',
 ('안드로이드 개발자 생각하고 있고 자격증은 필수인가요??', '앱 여러개 만들어보는게 제 스펙에 더 도움될거 같은데..'),
 ())

In [3]:
with open('answers.dic','rb') as fp:
    answers = pickle.load(fp) 

In [7]:
answers['#508581'] # 댓글접근, 이중dict으로 되어있으며 게시글 id에 인덱스를 붙여 댓글 id를 형성하였다

{'#508581_0': ('협군',
  '2018-10-07 01:01:07',
  '2',
  '정처기말고는 필요한 자격증이 없는데요.차라리 외국어 능력을 갖추세요.'),
 '#508581_1': ('에스에이치', '2018-10-07 01:29:47', '0', '수동보통1종뿐입니다만'),
 '#508581_2': ('code-01', '2018-10-07 11:45:19', '0', '필수 아닙니다.')}